In [10]:
from repsheet_backend.genai import anthropic, genai_cache
import asyncio

In [11]:
async def save_batch_to_cache(batch_id: str):
    results = await asyncio.to_thread(anthropic.messages.batches.results, batch_id)
    saved_count = 0
    cache_set_jobs = []
    for result in results:
        cache_key = result.custom_id
        # there are some old jobs with numeric custom_ids
        if len(cache_key) > 10:
            cache_set_jobs.append(genai_cache.set(cache_key, result.result.message.content[0].text))
            saved_count += 1
    await asyncio.gather(*cache_set_jobs)
    print(f"Saved {saved_count} results for {batch_id}")


In [12]:
await asyncio.gather(*[
    save_batch_to_cache(message.id)
    for message in anthropic.messages.batches.list()
    if message.processing_status == "ended"
])


Saved 0 results for msgbatch_012arDNFnmWQ77p1E2WLu7JM
Saved 0 results for msgbatch_01EciNEfCLG57CSof2mhJhSg
Saved 23 results for msgbatch_01FSZiaU3XfqusgjdLSvMoRv
Saved 23 results for msgbatch_01W6acgwuHBMY7UNCJRaFdAS
Saved 23 results for msgbatch_01UKCzgXfmJj9ndXE4AQm93a
Saved 23 results for msgbatch_01Gd4AcceMn8auBVp6GiDyB6
Saved 23 results for msgbatch_01Wz1ZivAxbgwwrNTuWQHJW9
Saved 23 results for msgbatch_014XT5dkYKoJeSb3ePQ5uwYt
Saved 23 results for msgbatch_0163NcVi7itBs5PKLgcD3CVw
Saved 23 results for msgbatch_01KToPWSon1UiDFDGFQBK1Uq
Saved 23 results for msgbatch_01DyJqmjDPoQRjLbdbSUPCsv
Saved 23 results for msgbatch_01Kmxwqcqig8UM27YdCeZZN9
Saved 23 results for msgbatch_01Rx3i9sXsPqM3JTsG4u94TF
Saved 23 results for msgbatch_01Bmz6YTCyzZBDNZgTiYRyt4
Saved 23 results for msgbatch_0123LN2mhy4GzdrgDZiD7uSa
Saved 23 results for msgbatch_01TJfTmAeqHaaG8s1C7zgBZE
Saved 23 results for msgbatch_016Do3BtAk9VYyWR5imKXY5F
Saved 23 results for msgbatch_01DWipQU53aKne4NSfH493L7
Saved 23 res

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,